# Potential Sites

## Imports

In [83]:
import urllib.request, json
import pandas as pd
import math
import pickle
import collections
from tqdm import tqdm
import geopandas
import datetime
import os
import numpy as np
import requests
import json

## Global Variables

In [84]:
LOCAL_AUTHORITY_ID = 120 # Luton = 120
YEAR = 2018
ROAD_NAME = 'M1'
TOTAL_PAGES = 3862

## General Functions

In [85]:
def get_aadt_by_direction(count_point_id=None, region_id=None, local_authority_id=None, year=None, count_method=None, road_name=None, road_category=None):
    url = "https://roadtraffic.dft.gov.uk/api/average-annual-daily-flow-by-direction"
    filters = {
        "count_point_id": count_point_id,
        "region_id": region_id,
        "local_authority_id": local_authority_id,
        "year": year,
        "count_method": count_method,
        "road_name": road_name,
        "road_category": road_category
    }
    params = {
        'filter': json.dumps(filters)
    }
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    return data

In [86]:
def get_aadt_by_direction_1(count_point_id=None, region_id=None, local_authority_id=None, year=None, count_method=None, road_name=None, road_category=None, page=1):
    url = "https://roadtraffic.dft.gov.uk/api/average-annual-daily-flow-by-direction"
    filters = {
        "count_point_id": count_point_id,
        "region_id": region_id,
        "local_authority_id": local_authority_id,
        "year": year,
        "count_method": count_method,
        "road_name": road_name,
        "road_category": road_category
    }
    params = {
        'filter': json.dumps(filters),
        'page': page
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to get AADF by direction: %s" % response.text)

In [87]:
def filter_dict_by_dict(dictionary, filter_dict):
    return {k: v for k, v in dictionary.items() if k in filter_dict and dictionary[k] == filter_dict[k]}

## Data Loading and Pre-Processing

In [91]:
aadt_list = [get_aadt_by_direction_1(local_authority_id=LOCAL_AUTHORITY_ID, year=YEAR, road_name=ROAD_NAME, page=page) for page in range(1, 5)]

aadt_by_direction_json = get_aadt_by_direction(local_authority_id=LOCAL_AUTHORITY_ID, year=YEAR, road_name=ROAD_NAME)

{'current_page': 1,
 'data': [{'id': 1,
   'count_point_id': 6834,
   'year': 2014,
   'region_id': 9,
   'local_authority_id': 81,
   'road_name': 'A259',
   'road_type': 'Major',
   'start_junction_road_name': 'A286',
   'end_junction_road_name': 'A27',
   'easting': 487050,
   'northing': 104670,
   'latitude': '50.83502701',
   'longitude': '-0.76519815',
   'link_length_km': '1.40',
   'link_length_miles': '0.87',
   'estimation_method': 'Estimated',
   'estimation_method_detailed': "Estimated using previous year's AADF on this link",
   'direction_of_travel': 'E',
   'pedal_cycles': 30,
   'two_wheeled_motor_vehicles': 73,
   'cars_and_taxis': 4499,
   'buses_and_coaches': 84,
   'lgvs': 713,
   'hgvs_2_rigid_axle': 55,
   'hgvs_3_rigid_axle': 4,
   'hgvs_4_or_more_rigid_axle': 0,
   'hgvs_3_or_4_articulated_axle': 0,
   'hgvs_5_articulated_axle': 1,
   'hgvs_6_articulated_axle': 3,
   'all_hgvs': 63,
   'all_motor_vehicles': 5432},
  {'id': 2,
   'count_point_id': 6834,
   'year

In [ ]:
filters = {'local_authority_id': LOCAL_AUTHORITY_ID, 'year': YEAR, 'road_name': ROAD_NAME}

aadt_filtered = filter_dict_by_dict(aadt_by_direction_json, filters)

aadt_filtered

{}

## Potential Sites Data Loading and Pre-Processing

In [ ]:
df_luton = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_120.csv')
df_worcester = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_69.csv')
df_hounslow = pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_111.csv')